In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'soil-types-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3653989%2F6345760%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240922%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240922T161143Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8c154a89c932d1d82947272588dc43b617b61c29cdc2319e69ffe87ff18e5288d6cbbe796cd6864851539baaa84ab7bf411847e51914888119474bfd674b80e431e6b51ed3bf3aedb8d549f8e6f762c2a3ee641331fba1c02f1fae0b66745b30d676afe3cf7d510cb5f7613f03d3a21fc818c4b10eb4629be8d7ed17956e4f50573d9d9616e0ff698a2f6839aa755b8524b8eeac1be2ee409eeb250d066ae9d8622b46113440dd416a2289ad1fbf63330add4d0557a5f998ec1b00579ee388d3d1ef4e25ef9425251fb572b65bf5ae4513a755ccc924be36bb365039b79297dc85147ea1fad0ecbe3296431b633458d8f12e6ec52da2933c37fc8e7a06124be4,cropsky/keras/default/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F92621%2F110558%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240922%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240922T161143Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db3c0dc6c513de97a20dbf77b1e6307d023a239cd466d74ed5262c395bd42259cb316f754a85eafde3cf5550a0bbb794cf271e47b5226559c6b222b4703eef6f8aa9dc5122cf43c36ffdfffa8d71675f26deb195f560f04f0e0e403656cbbcbb8a8f1b5c4503d71ee95a8c26b4c903dc86f7b16c1c7653d82e02422784fac679cda69d364c2300ab0223bc435ed9f255a803b84afa7df5b3ef08babbad4697d07ec7ff6c980eb1c56a1def50e139676b8d31843255b04f4d87638af8673b5e6655dc4033435b6e57c00f3dec11abc96cf17f8aea31e3619b8f45c2da40bc26bd97aa31f75300d0ccf119a66c746749b4cdc4e4fc47c5305afed98546a0184536c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 207384505 bytes downloaded
Downloaded and uncompressed: soil-types-dataset
[==================================================] 169813380 bytes downloaded
Downloaded and uncompressed: cropsky/keras/default/1
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Copy of images104.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Copy of images135.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Red_23.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Red_42.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Copy of images.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Copy of image23.jpeg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Red_14.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Copy of images105.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Copy of image3.jpeg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Copy of download (2).jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Red_32.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Red_21.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Red_38.jpg
/kaggle/input/soil-types-dataset/Dataset/test/Red soil/Copy of image

In [ ]:
import re
import os
import cv2
import time
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip= True,
    rotation_range= 10,
    shear_range= 0.2,
    height_shift_range= 0.1,
    width_shift_range= 0.1)

In [ ]:
train_gen=image_gen.flow_from_directory(directory='/kaggle/input/soil-types-dataset/Dataset/Train',seed=42,target_size=(224,224),batch_size=32)
val_gen=image_gen.flow_from_directory(directory='/kaggle/input/soil-types-dataset/Dataset/test',seed=42,target_size=(224,224),batch_size=32)

Found 1721 images belonging to 7 classes.
Found 395 images belonging to 7 classes.


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
model=tf.keras.applications.VGG16(include_top=False,input_shape=(224,224,3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.output_shape

(None, 7, 7, 512)

In [ ]:
for layer in model.layers[:-2]:
    layer.trainable = False

In [ ]:
x=tf.keras.layers.Flatten()(model.output)
x=tf.keras.layers.Dense(512,activation='relu')(x)
x=tf.keras.layers.Dropout(0.2)(x)
output=tf.keras.layers.Dense(7,activation='softmax')(x)

In [ ]:
fin_model=tf.keras.Model(inputs=model.input,outputs=output)

In [ ]:
fin_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

In [ ]:
history=fin_model.fit(train_gen,epochs=15,validation_data=val_gen)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


54/54 ━━━━━━━━━━━━━━━━━━━━ 71s 956ms/step - accuracy: 0.4118 - loss: 2.4407 - val_accuracy: 0.6734 - val_loss: 0.9478
Epoch 2/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 38s 637ms/step - accuracy: 0.7278 - loss: 0.7732 - val_accuracy: 0.7038 - val_loss: 0.7633
Epoch 3/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 34s 570ms/step - accuracy: 0.7801 - loss: 0.5808 - val_accuracy: 0.8101 - val_loss: 0.4527
Epoch 4/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 558ms/step - accuracy: 0.8329 - loss: 0.4675 - val_accuracy: 0.8177 - val_loss: 0.4972
Epoch 5/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 567ms/step - accuracy: 0.8511 - loss: 0.3919 - val_accuracy: 0.8354 - val_loss: 0.4405
Epoch 6/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 40s 564ms/step - accuracy: 0.8832 - loss: 0.3193 - val_accuracy: 0.8835 - val_loss: 0.2927
Epoch 7/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 42s 587ms/step - accuracy: 0.8926 - loss: 0.2910 - val_accuracy: 0.8759 - val_loss: 0.3215
Epoch 8/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 567ms/step - accuracy: 0.8904 - loss: 0.2840 - val_accuracy: 0.926

In [ ]:
z= fin_model.evaluate(val_gen)

13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 497ms/step - accuracy: 0.9214 - loss: 0.1697


In [ ]:
fin_model.save('Cropsky_7soils.h5')

In [ ]:
print(z)